<header>
        <div style="overflow: auto;">
        <img src="./images/AMS_LogoRED3.png" style="float: left;" width="300" />
        <img src="./images/TUDelft.jpg" style="float: right;" />
    </div>
    <div style="text-align: center;">
        <h1> AMS Data1 </h1>
        <h2>Workshop 7: Pretty Plots in Python</h2>
        <h6>&copy; 2023, TU Delft. Hans Hoogenboom, Creative Commons</h6>     
    </div>
    <br>   
    <br>
</header>

## Folium

Folium is a Python library based on the [Leaflet.js](https://leafletjs.com/) Javascript library. Both are popular libraries for creating tile based maps. [Tile based](https://en.wikipedia.org/wiki/Tiled_web_map) maps are an alternative to methods like Web Map Services (WMS), a protocol used heavily by PDOK.



In [ ]:
import folium
m = folium.Map(location=(52.3136, 4.8250))
m

By default Folium will use the OpenStreetMap tile set but you can choose a different one by specifying the option *tile="something"* to the *Map* method. Foloium supports the following tile sets:

* OpenStreetMap
* Mapbox
* Mapbox Bright
* Mapbox Control Room
* Stamen, Terrain, Toner, Watercolor
* Cloudmade
* CartoDB, positron, dark_matter

The Mapbox service needs an API Key (can be requested on their site) and the two other Mapbox tiles have limited functionality (zoom levels and number of tiles)




In [ ]:
m_lower = folium.Map(location=(43.3136, 4.8250), tiles="CartoDB positron")
m_lower

As you might expect there are a lot more options to control the display of the map. Below we create a map with the AMS as center and locked scrolling and panning. Documentation can be found [here](https://python-visualization.github.io/folium/latest/reference.html#module-folium.folium)

In [ ]:
AMS = folium.Map(location=[52.3737966,4.9148386],
               zoom_start=5,
               zoom_control=False,
               scrollWheelZoom=False,
               dragging=False)
AMS

You can add markers to a map using the *Markers* method. Let's add all the AirBnB locations from Amsterdam Centrum-West on the map. We open the listings.csv and create a Geopandas dataframe. We only want the entries with the value *Centrum-West* in the neighbourhood column. We create a list with *Shapely* points bsaed on the longitude and latitude. For the markers on the map we don't want all the data, just a few columns.

In [ ]:
import geopandas

airbnb = geopandas.read_file('./data/listings.csv')
print(airbnb.columns)
df_west = airbnb.loc[airbnb['neighbourhood'] == "Centrum-West"]
geometry = geopandas.points_from_xy(df_west.longitude, df_west.latitude)
df_plot_data = geopandas.GeoDataFrame(df_west[['id', 'host_name', 'latitude', 'longitude', 'price']], geometry=geometry)
df_plot_data.head()

We create a new AMS map with some other options that control the map. We zoomed in a bit more, set the width and height of the frame of the map and added a scale bar.

In [ ]:
AMS = folium.Map(location=(52.3737966,4.9148386), width='80%', height='80%', zoom_start=14, control_scale=True)
AMS

The points we created are not yet readable by Folium so we need to convert them to something a bit more simple. Let's examine one point:

In [ ]:
point = df_plot_data.iloc[2, df_plot_data.columns.get_loc('geometry')]
print(type(point))
print(point)

In the documentation of [Shapely]('https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html#shapely.Point') we can find what we can do with the point object. The *xy* method will seperate the coordinates of a point into seperate values. We create a simple list of the coordinates which we can use later as coordinate for the *markers*.

In [ ]:
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in df_plot_data.geometry]
#print(geo_df_list)

In [ ]:
for i, row in df_plot_data.iterrows():
    print(i)

The [*Marker*](https://python-visualization.github.io/folium/latest/reference.html#folium.map.Marker) method in Folium let's you create points on the map. It needs a location and optionally a [*popup*](https://python-visualization.github.io/folium/latest/reference.html#folium.map.Popup), text to display, a tooltip and an icon. We iterate/loop over the data and the marker element to the map using the *add_child* method.

In [ ]:
i = 0
for row_index, row in df_plot_data.iterrows():
    mark =  folium.Marker(
                location=geo_df_list[i],
                popup=(
                "Name: "
                + str(df_plot_data.host_name[row_index])
                + "<br>" +
                "Id: "
                + str(df_plot_data.id[row_index])
                + "<br>" +
                "Price: "
                + str(df_plot_data.price[row_index])
                #icon=folium.Icon(color="%s" % type_color),
                )
            )
    i += 1
    AMS.add_child(mark)

We added all the data to the map so now we can display it:

In [ ]:
AMS

## Links
[Folium site](https://python-visualization.github.io/folium/latest/index.html)

[GeoPandas and Folium](https://geopandas.org/en/stable/gallery/plotting_with_folium.html)

[Medium Introduction](https://medium.com/datasciencearth/map-visualization-with-folium-d1403771717)

Hans Hoogenboom</br>
Design Informatics</br>
2023 &copy;